# **Installs** the three LangChain packages (langchain_groq, langchain_core, langchain_community) so the notebook can call Groq’s LLM and use LangChain utilities.

In [ ]:
!pip install langchain_groq langchain_core langchain_community

*  Imports ChatGroq from langchain_groq.

* Creates an LLM instance with your API key and the “llama-3.3-70b-versatile” model.


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "YOUR_GROK_API_KEY",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("You are a mental health chatbot, try to provide assistance for the user and make them feel comfortable.")
print(result.content)

**Installs pypdf; this lightweight library lets later cells read and parse PDF files that will be fed into the vector database.**

In [ ]:
!pip install pypdf

**Installs chromadb plus its many dependencies; Chroma is the local vector store used to hold chunk embeddings so the chatbot can retrieve contextual passages at run-time.**

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers


**Imports everything the rest of the notebook needs**

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os



**Wraps the ChatGroq constructor in a helper so you can spin up the same LLM configuration anywhere with one call.**

In [ ]:
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "YOUR_GROK_API_KEY",
    model_name = "llama-3.3-70b-versatile"
)
  return llm


**Loads every PDF inside /content/data/.**

**Splits each document into 500-character chunks with 50-char overlap.**

**Embeds the chunks with all-MiniLM-L6-v2.**

**Stores the resulting vectors in a persistent Chroma database at ./chroma_db and tells you when it’s done.**

In [ ]:
def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db



Takes a Chroma retriever and LLM, then builds a LangChain RetrievalQA pipeline that:
– Inserts retrieved context plus the user’s question into a custom prompt (“You are a compassionate mental health chatbot…”)
– Lets the LLM generate the final answer.

In [ ]:
def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain




Checks whether a Chroma database already exists; if not, it builds one from the PDFs with create_vector_db(); otherwise it reloads the stored vectors.

Builds the Retrieval-QA pipeline by passing the vector store and LLM to setup_qa_chain().

Enters an infinite chat loop: waits for user text, breaks cleanly on exit, or pipes any other query through qa_chain.run() and prints the chatbot’s response

In [ ]:
# os.makedirs('/content/data/', exist_ok=True)
# create_vector_db()

In [ ]:
def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

In [ ]:
!pip install --quiet gradio


In [ ]:

import gradio as gr

llm = initialize_llm()
db_path = "/content/chroma_db"

if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

def answer(user_message, chat_history):
    return qa_chain.run(user_message)

demo = gr.ChatInterface(
    fn=answer,
    title="🧠 Mental-Health PDF Chatbot",
    description="Ask anything about the uploaded PDFs — type *exit* in the CLI or just close the tab to quit.",
)
demo.launch()


In [ ]:
!pip install spacy textblob langchain-experimental
!python -m spacy download en_core_web_sm

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
import spacy

class MultiAgentMentalHealthChatbot:
    def __init__(self):
        self.llm = self.initialize_llm()
        self.vector_db = self.load_vector_db()
        self.nlp = spacy.load("en_core_web_sm")
        self.memory = ConversationBufferMemory()

        # Initialize specialized agents
        self.retrieval_agent = self.create_retrieval_agent()
        self.emotion_agent = self.create_emotion_agent()
        self.response_agent = self.create_response_agent()
        self.safety_agent = self.create_safety_agent()

    def create_retrieval_agent(self):
        def retrieve_context(query: str) -> str:
            retriever = self.vector_db.as_retriever(search_kwargs={"k": 3})
            docs = retriever.get_relevant_documents(query)
            return "\n".join([doc.page_content for doc in docs])

        return Tool(
            name="knowledge_retrieval",
            description="Retrieves relevant mental health information from knowledge base",
            func=retrieve_context
        )

    def create_emotion_agent(self):
        def analyze_emotion(text: str) -> dict:
            doc = self.nlp(text)

            # Extract emotional indicators
            emotions = {
                'sentiment': self.get_sentiment_score(doc),
                'emotional_words': [token.lemma_ for token in doc if token.pos_ in ['ADJ', 'VERB'] and token.sentiment != 0],
                'intensity': self.calculate_emotional_intensity(doc),
                'crisis_indicators': self.detect_crisis_words(doc)
            }
            return emotions

        return Tool(
            name="emotion_analysis",
            description="Analyzes emotional state and sentiment from user input",
            func=lambda x: str(analyze_emotion(x))
        )


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from textblob import TextBlob

class EmotionProcessor:
    def __init__(self):
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except OSError:
            # Download model if not available
            spacy.cli.download("en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

        # Add sentiment component
        if "textcat" not in self.nlp.pipe_names:
            self.nlp.add_pipe("textcat")

    def process_text(self, text: str) -> dict:
        doc = self.nlp(text)

        # Lemmatization with filtering
        lemmatized_tokens = [
            token.lemma_.lower()
            for token in doc
            if not token.is_stop
            and not token.is_punct
            and not token.is_space
            and len(token.text) > 2
        ]

        # Emotional analysis
        emotional_analysis = {
            'lemmatized_text': ' '.join(lemmatized_tokens),
            'entities': [(ent.text, ent.label_) for ent in doc.ents],
            'sentiment': self.get_sentiment(text),
            'emotional_keywords': self.extract_emotional_keywords(doc),
            'pos_tags': [(token.text, token.pos_) for token in doc]
        }

        return emotional_analysis

    def get_sentiment(self, text: str) -> dict:
        blob = TextBlob(text)
        return {
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity,
            'classification': self.classify_sentiment(blob.sentiment.polarity)
        }

    def classify_sentiment(self, polarity: float) -> str:
        if polarity > 0.1:
            return "positive"
        elif polarity < -0.1:
            return "negative"
        else:
            return "neutral"

    def extract_emotional_keywords(self, doc) -> list:
        emotional_words = []
        emotion_lexicon = {
            'anxiety': ['anxious', 'worried', 'nervous', 'stressed'],
            'depression': ['sad', 'depressed', 'hopeless', 'empty'],
            'anger': ['angry', 'frustrated', 'irritated', 'mad'],
            'joy': ['happy', 'excited', 'joyful', 'content']
        }

        for token in doc:
            lemma = token.lemma_.lower()
            for emotion, words in emotion_lexicon.items():
                if lemma in words:
                    emotional_words.append((lemma, emotion))

        return emotional_words


**RAG IMPLEMENTATION**

In [ ]:
import os
import spacy
from textblob import TextBlob
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
import gradio as gr

class EmotionProcessor:
    def __init__(self):
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except OSError:
            # Download model if not available
            os.system("python -m spacy download en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

    def process_text(self, text: str) -> dict:
        doc = self.nlp(text)

        # Lemmatization with filtering
        lemmatized_tokens = [
            token.lemma_.lower()
            for token in doc
            if not token.is_stop
            and not token.is_punct
            and not token.is_space
            and len(token.text) > 2
        ]

        # Emotional analysis
        emotional_analysis = {
            'lemmatized_text': ' '.join(lemmatized_tokens),
            'entities': [(ent.text, ent.label_) for ent in doc.ents],
            'sentiment': self.get_sentiment(text),
            'emotional_keywords': self.extract_emotional_keywords(doc),
            'pos_tags': [(token.text, token.pos_) for token in doc]
        }

        return emotional_analysis

    def get_sentiment(self, text: str) -> dict:
        blob = TextBlob(text)
        return {
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity,
            'classification': self.classify_sentiment(blob.sentiment.polarity)
        }

    def classify_sentiment(self, polarity: float) -> str:
        if polarity > 0.1:
            return "positive"
        elif polarity < -0.1:
            return "negative"
        else:
            return "neutral"

    def extract_emotional_keywords(self, doc) -> list:
        emotional_words = []
        emotion_lexicon = {
            'anxiety': ['anxious', 'worried', 'nervous', 'stressed'],
            'depression': ['sad', 'depressed', 'hopeless', 'empty'],
            'anger': ['angry', 'frustrated', 'irritated', 'mad'],
            'joy': ['happy', 'excited', 'joyful', 'content']
        }

        for token in doc:
            lemma = token.lemma_.lower()
            for emotion, words in emotion_lexicon.items():
                if lemma in words:
                    emotional_words.append((lemma, emotion))

        return emotional_words

class MentalHealthMultiAgent:
    def __init__(self):
        self.setup_components()
        self.create_tools()
        self.setup_qa_chain()

    def setup_components(self):
        # Initialize LLM
        self.llm = ChatGroq(
            temperature=0,
            groq_api_key="YOUR_GROK_API_KEY",
            model_name="llama-3.3-70b-versatile"
        )

        # Load or create vector database
        self.vector_db = self.load_vector_db()

        # Initialize emotion processor
        self.emotion_processor = EmotionProcessor()

        # Initialize memory
        self.conversation_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

    def load_vector_db(self):
        db_path = "./chroma_db"

        if not os.path.exists(db_path):
            return self.create_vector_db()
        else:
            embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
            return Chroma(persist_directory=db_path, embedding_function=embeddings)

    def create_vector_db(self):
        # Create data directory if it doesn't exist
        os.makedirs('./data/', exist_ok=True)

        loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
        documents = loader.load()

        if not documents:
            print("No PDF documents found in ./data/ directory")
            # Create empty vector store
            embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
            vector_db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)
            return vector_db

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        texts = text_splitter.split_documents(documents)
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
        vector_db.persist()

        print("ChromaDB created and data saved")
        return vector_db

    def create_tools(self):
        self.tools = [
            Tool(
                name="retrieve_knowledge",
                description="Search mental health knowledge base for relevant information",
                func=self.retrieve_knowledge
            ),
            Tool(
                name="analyze_emotion",
                description="Analyze emotional state and sentiment from user input",
                func=self.analyze_emotion
            ),
            Tool(
                name="check_safety",
                description="Check for crisis indicators or safety concerns",
                func=self.check_safety
            )
        ]

    def setup_qa_chain(self):
        retriever = self.vector_db.as_retriever()
        prompt_template = """You are a compassionate mental health chatbot. Use the following context and emotional analysis to respond thoughtfully:

Context: {context}
Emotional Analysis: {emotion_analysis}
Safety Check: {safety_status}

User: {question}
Chatbot: """

        self.PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=['context', 'emotion_analysis', 'safety_status', 'question']
        )

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": self.PROMPT}
        )

    def retrieve_knowledge(self, query: str) -> str:
        try:
            retriever = self.vector_db.as_retriever(search_kwargs={"k": 3})
            docs = retriever.get_relevant_documents(query)
            context = "\n".join([doc.page_content for doc in docs])
            return f"Retrieved context: {context}" if context else "No relevant context found"
        except Exception as e:
            return f"Error retrieving knowledge: {str(e)}"

    def analyze_emotion(self, text: str) -> str:
        try:
            analysis = self.emotion_processor.process_text(text)
            return f"Emotional analysis: {analysis}"
        except Exception as e:
            return f"Error analyzing emotion: {str(e)}"

    def check_safety(self, text: str) -> str:
        crisis_keywords = ['suicide', 'kill myself', 'end it all', 'hurt myself', 'want to die']
        text_lower = text.lower()

        crisis_detected = any(keyword in text_lower for keyword in crisis_keywords)

        if crisis_detected:
            return "CRISIS DETECTED: Please seek immediate professional help. Contact a crisis hotline or emergency services."
        return "No immediate safety concerns detected"

    def chat(self, user_input: str) -> str:
        try:
            # Get context from knowledge base
            context = self.retrieve_knowledge(user_input)

            # Analyze emotions
            emotion_analysis = self.analyze_emotion(user_input)

            # Check safety
            safety_status = self.check_safety(user_input)

            # If crisis detected, return immediate response
            if "CRISIS DETECTED" in safety_status:
                return safety_status + "\n\nI'm here to support you, but please reach out to a mental health professional or crisis hotline immediately. Your safety is the most important thing."

            # Generate response using the QA chain
            response = self.llm.invoke(f"""You are a compassionate mental health chatbot.

Context from knowledge base: {context}
Emotional analysis: {emotion_analysis}
Safety status: {safety_status}

User message: {user_input}

Please provide a thoughtful, empathetic response that acknowledges the user's emotional state and provides helpful guidance.""")

            return response.content

        except Exception as e:
            return f"I'm sorry, I encountered an issue processing your message. Please try rephrasing your question. Error: {str(e)}"

# Gradio Interface
def create_gradio_interface():
    chatbot = MentalHealthMultiAgent()

    def respond(user_message, chat_history):
        bot_response = chatbot.chat(user_message)
        return bot_response

    demo = gr.ChatInterface(
        fn=respond,
        title="🧠 Multi-Agent Mental Health Chatbot",
        description="An advanced mental health chatbot with emotion analysis, safety monitoring, and knowledge retrieval capabilities.",
        examples=[
            "I'm feeling anxious about my upcoming exam",
            "I've been feeling really sad lately",
            "Can you help me with stress management techniques?",
            "I'm having trouble sleeping"
        ]
    )

    return demo

# Command Line Interface
def main():
    print("Initializing Multi-Agent Mental Health Chatbot...")
    chatbot = MentalHealthMultiAgent()

    print("Multi-Agent Mental Health Chatbot initialized!")
    print("Type 'exit' to quit, 'gradio' to launch web interface\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Take care! Remember, professional help is always available.")
            break
        elif user_input.lower() == 'gradio':
            print("Launching Gradio interface...")
            demo = create_gradio_interface()
            demo.launch()
            break

        response = chatbot.chat(user_input)
        print(f"Chatbot: {response}\n")

if __name__ == "__main__":
    # Install required packages
    try:
        import spacy
        spacy.load("en_core_web_sm")
    except:
        print("Installing required spaCy model...")
        os.system("pip install spacy textblob")
        os.system("python -m spacy download en_core_web_sm")

    main()


In [ ]:
!pip install langchain langgraph langchain-core langchain-community


In [ ]:
!pip install crewai spacy textblob gradio langchain-groq chromadb sentence-transformers langchain-experimental
!python -m spacy download en_core_web_sm


**CREW AI IMPLEMENTATION WITH RAG**

In [ ]:
!pip install langchain langchain-core langchain-experimental
!pip install -U langchain-community
!pip install langchain-community langchain-groq
!pip install crewai spacy textblob gradio chromadb sentence-transformers pydantic
import os
import spacy
from textblob import TextBlob
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# import gradio as gr

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from typing import Type, Any
from pydantic import BaseModel, Field

# Global variables to store shared resources
VECTOR_DB = None
NLP_MODEL = None

def initialize_nlp():
    """Initialize spaCy model globally"""
    global NLP_MODEL
    if NLP_MODEL is None:
        try:
            NLP_MODEL = spacy.load("en_core_web_sm")
        except OSError:
            os.system("python -m spacy download en_core_web_sm")
            NLP_MODEL = spacy.load("en_core_web_sm")
    return NLP_MODEL

def load_existing_vector_db():
    """Load existing ChromaDB from persisted directory"""
    global VECTOR_DB

    db_path = "./chroma_db"

    # Always try to load existing database first
    if os.path.exists(db_path):
        try:
            embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
            VECTOR_DB = Chroma(
                persist_directory=db_path,
                embedding_function=embeddings
            )

            # Check if database has content
            try:
                collection_data = VECTOR_DB.get()
                collection_count = len(collection_data['documents'])
                print(f"Loaded existing ChromaDB with {collection_count} documents")

                if collection_count == 0:
                    print("Warning: ChromaDB exists but contains no documents")
                    print("Attempting to load documents from ./data/ folder...")
                    return create_vector_db_from_data()
                else:
                    return VECTOR_DB

            except Exception as e:
                print(f"Error checking database content: {str(e)}")
                return create_vector_db_from_data()

        except Exception as e:
            print(f"Error loading existing ChromaDB: {str(e)}")
            print("Creating new vector database...")
            return create_vector_db_from_data()
    else:
        print("No existing ChromaDB found. Creating new one...")
        return create_vector_db_from_data()

def create_vector_db_from_data():
    """Create new ChromaDB from PDF documents in data folder"""
    global VECTOR_DB

    # Ensure data directory exists
    os.makedirs('./data/', exist_ok=True)


    # Load documents from data folder
    loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()

    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    if not documents:
        print("No PDF documents found in ./data/ directory")
        print("Creating empty vector store...")
        VECTOR_DB = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)
        return VECTOR_DB

    print(f"Found {len(documents)} PDF documents. Processing...")

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    print(f"Split into {len(texts)} text chunks")

    # Create vector database
    VECTOR_DB = Chroma.from_documents(
        texts,
        embeddings,
        persist_directory='./chroma_db'
    )

    # Ensure persistence (for older ChromaDB versions)
    try:
        VECTOR_DB.persist()
        print("Database persisted successfully")
    except AttributeError:
        print("Auto-persistence enabled")

    print(f"ChromaDB created with {len(texts)} document chunks and saved to ./chroma_db")
    return VECTOR_DB

# Custom Tools for CrewAI (rest of your tool code remains the same)
class KnowledgeRetrievalInput(BaseModel):
    query: str = Field(description="The query to search for in the knowledge base")

class KnowledgeRetrievalTool(BaseTool):
    name: str = "knowledge_retrieval"
    description: str = "Retrieves relevant mental health information from the knowledge base"
    args_schema: Type[BaseModel] = KnowledgeRetrievalInput

    def _run(self, query: str) -> str:
        global VECTOR_DB
        try:
            if VECTOR_DB is None:
                return "Knowledge base not initialized"

            retriever = VECTOR_DB.as_retriever(search_kwargs={"k": 3})
            docs = retriever.get_relevant_documents(query)
            context = "\n".join([doc.page_content for doc in docs])
            return f"Retrieved context: {context}" if context else "No relevant context found"
        except Exception as e:
            return f"Error retrieving knowledge: {str(e)}"

# Emotion Analysis Tool
class EmotionAnalysisInput(BaseModel):
    text: str = Field(description="The text to analyze for emotional content")

class EmotionAnalysisTool(BaseTool):
    name: str = "emotion_analysis"
    description: str = "Analyzes emotional state and sentiment from user input"
    args_schema: Type[BaseModel] = EmotionAnalysisInput

    def _run(self, text: str) -> str:
        try:
            nlp = spacy.load("en_core_web_sm")
            doc = nlp(text)
            blob = TextBlob(text)

            # Extract emotional indicators
            lemmatized_tokens = [
                token.lemma_.lower()
                for token in doc
                if not token.is_stop and not token.is_punct and len(token.text) > 2
            ]

            emotion_lexicon = {
                'anxiety': ['anxious', 'worried', 'nervous', 'stressed', 'panic'],
                'depression': ['sad', 'depressed', 'hopeless', 'empty', 'worthless'],
                'anger': ['angry', 'frustrated', 'irritated', 'mad', 'furious'],
                'joy': ['happy', 'excited', 'joyful', 'content', 'pleased']
            }

            detected_emotions = []
            for emotion, words in emotion_lexicon.items():
                for token in lemmatized_tokens:
                    if token in words:
                        detected_emotions.append(emotion)

            analysis = {
                'sentiment_polarity': blob.sentiment.polarity,
                'sentiment_subjectivity': blob.sentiment.subjectivity,
                'detected_emotions': list(set(detected_emotions)),
                'emotional_intensity': 'high' if abs(blob.sentiment.polarity) > 0.5 else 'moderate' if abs(blob.sentiment.polarity) > 0.2 else 'low'
            }

            return f"Emotional analysis: {analysis}"
        except Exception as e:
            return f"Error analyzing emotion: {str(e)}"

# Safety Monitor Tool
class SafetyMonitorInput(BaseModel):
    text: str = Field(description="The text to monitor for safety concerns")

class SafetyMonitorTool(BaseTool):
    name: str = "safety_monitor"
    description: str = "Monitors for crisis indicators and safety concerns"
    args_schema: Type[BaseModel] = SafetyMonitorInput

    def _run(self, text: str) -> str:
        crisis_keywords = [
            'suicide', 'kill myself', 'end it all', 'hurt myself', 'want to die',
            'self harm', 'cut myself', 'overdose', 'jump off', 'hang myself'
        ]

        urgent_keywords = [
            'emergency', 'crisis', 'help me', 'cant take it', 'breaking point'
        ]

        text_lower = text.lower()

        crisis_detected = any(keyword in text_lower for keyword in crisis_keywords)
        urgent_detected = any(keyword in text_lower for keyword in urgent_keywords)

        if crisis_detected:
            return "CRISIS_ALERT: Immediate suicide risk detected. Emergency intervention required."
        elif urgent_detected:
            return "URGENT_CONCERN: High distress level detected. Immediate support recommended."
        else:
            return "SAFE: No immediate safety concerns detected."

class CrewAIMentalHealthChatbot:
    def __init__(self):
        self.setup_llm()
        self.setup_vector_db()
        self.setup_tools()
        self.setup_agents()

    def setup_llm(self):
        self.llm = LLM(
            model="groq/llama-3.3-70b-versatile",
            api_key="YOUR_GROK_API_KEY",
            temperature=0.3
        )

    def setup_vector_db(self):
        global VECTOR_DB
        # Load existing database or create new one from data
        VECTOR_DB = load_existing_vector_db()

        # Verify the database is working
        if VECTOR_DB is not None:
            try:
                test_docs = VECTOR_DB.get()
                doc_count = len(test_docs['documents'])
                print(f"Vector database ready with {doc_count} documents")

                if doc_count == 0:
                    print("Note: Vector database is empty. Add PDF files to ./data/ folder and restart.")

            except Exception as e:
                print(f"Warning: Vector database may not be functioning properly: {str(e)}")

    def create_vector_db(self):
        # uncomment the below 4 lines when running the notebook for the first time in colab
        os.system("git clone https://github.com/DeepakAC3/CAMIT-Intern-MentalHealthChatbot/")
        os.system("cd CAMIT-Intern-MentalHealthChatbot/")
        os.system("cp -r CAMIT-Intern-MentalHealthChatbot/data/ .")
        os.system("rm -rf CAMIT-Intern-MentalHealthChatbot/")

        loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
        documents = loader.load()

        if not documents:
            print("No PDF documents found in ./data/ directory")
            embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
            vector_db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)
            return vector_db

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        texts = text_splitter.split_documents(documents)
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
        vector_db.persist()

        print("ChromaDB created and data saved")
        return vector_db

    def setup_tools(self):
        # Initialize spaCy model before creating tools
        initialize_nlp()

        self.knowledge_tool = KnowledgeRetrievalTool()
        self.emotion_tool = EmotionAnalysisTool()
        self.safety_tool = SafetyMonitorTool()

    def setup_agents(self):
        # Emotional Intelligence Agent
        self.emotion_agent = Agent(
            role='Emotional Intelligence Specialist',
            goal='Analyze user emotions and psychological state accurately',
            backstory="""You are an expert in emotional intelligence and psychological assessment.
            Your role is to carefully analyze user messages for emotional content, sentiment,
            and psychological indicators. You provide detailed emotional assessments that help
            other agents understand the user's mental state.""",
            tools=[self.emotion_tool],
            llm=self.llm,
            verbose=True,
            allow_delegation=False
        )

        # Knowledge Retrieval Agent
        self.knowledge_agent = Agent(
            role='Mental Health Knowledge Specialist',
            goal='Retrieve and synthesize relevant mental health information',
            backstory="""You are a mental health knowledge specialist with access to
            comprehensive mental health resources. Your role is to find and synthesize
            relevant information from the knowledge base to support therapeutic conversations.""",
            tools=[self.knowledge_tool],
            llm=self.llm,
            verbose=True,
            allow_delegation=False
        )

        # Safety Monitor Agent
        self.safety_agent = Agent(
            role='Crisis Intervention Specialist',
            goal='Monitor for safety risks and crisis situations',
            backstory="""You are a crisis intervention specialist responsible for identifying
            immediate safety risks and crisis situations. Your primary concern is user safety,
            and you must flag any concerning content that requires immediate attention.""",
            tools=[self.safety_tool],
            llm=self.llm,
            verbose=True,
            allow_delegation=False
        )

        # Therapeutic Response Agent
        self.therapist_agent = Agent(
            role='Compassionate Mental Health Therapist',
            goal='Provide empathetic, therapeutic responses based on analysis from other agents',
            backstory="""You are a compassionate mental health therapist with expertise in
            various therapeutic approaches. You synthesize information from emotional analysis,
            knowledge retrieval, and safety assessments to provide thoughtful, empathetic,
            and therapeutically appropriate responses to users.""",
            tools=[],
            llm=self.llm,
            verbose=True,
            allow_delegation=False
        )

    def chat(self, user_input: str) -> str:
        try:
            # Define tasks for each agent
            emotion_task = Task(
                description=f"Use the emotion_analysis tool to analyze this message: '{user_input}'. Provide detailed emotional insights.",
                agent=self.emotion_agent,
                expected_output="Detailed emotional analysis including sentiment, detected emotions, and intensity level"
            )

            knowledge_task = Task(
                description=f"Use the knowledge_retrieval tool to find relevant mental health information for: '{user_input}'. Focus on therapeutic guidance.",
                agent=self.knowledge_agent,
                expected_output="Relevant mental health information and therapeutic guidance from knowledge base"
            )

            safety_task = Task(
                description=f"Use the safety_monitor tool to assess this message for crisis indicators: '{user_input}'. Report any safety concerns.",
                agent=self.safety_agent,
                expected_output="Safety assessment with risk level and any crisis alerts"
            )

            therapy_task = Task(
                description=f"""Based on the previous analyses, provide a compassionate therapeutic response to: '{user_input}'.

                Consider the emotional state, relevant knowledge, and safety concerns to craft an appropriate response.
                Be warm, empathetic, and provide helpful guidance while validating the user's feelings.""",
                agent=self.therapist_agent,
                expected_output="Compassionate therapeutic response addressing the user's needs",
                context=[emotion_task, knowledge_task, safety_task]
            )

            # Create crew with tasks
            task_crew = Crew(
                agents=[self.emotion_agent, self.knowledge_agent, self.safety_agent, self.therapist_agent],
                tasks=[emotion_task, knowledge_task, safety_task, therapy_task],
                process=Process.sequential,
                verbose=False
            )

            # Execute the crew workflow
            result = task_crew.kickoff()

            # Extract the final therapeutic response
            if hasattr(result, 'raw'):
                return result.raw
            else:
                return str(result)

        except Exception as e:
            return f"I apologize, but I encountered an issue processing your message. Please try again. Error: {str(e)}"

# # Gradio Interface
# def create_gradio_interface():
#     print("Initializing CrewAI Mental Health Chatbot...")
#     chatbot = CrewAIMentalHealthChatbot()

#     def respond(user_message, chat_history):
#         bot_response = chatbot.chat(user_message)
#         return bot_response

#     demo = gr.ChatInterface(
#         fn=respond,
#         title="🧠 CrewAI Multi-Agent Mental Health Chatbot",
#         description="""
#         An advanced mental health chatbot powered by CrewAI multi-agent workflow:
#         • **Emotion Agent**: Analyzes your emotional state
#         • **Knowledge Agent**: Retrieves relevant mental health information
#         • **Safety Agent**: Monitors for crisis situations
#         • **Therapist Agent**: Provides compassionate therapeutic responses
#         """,
#         examples=[
#             "I'm feeling overwhelmed with work stress",
#             "I've been having trouble sleeping and feeling anxious",
#             "Can you help me understand depression better?",
#             "I'm going through a difficult breakup"
#         ],
#         theme="soft"
#     )

#     return demo

# Command Line Interface
def main():
    print("Initializing CrewAI Multi-Agent Mental Health Chatbot...")
    chatbot = CrewAIMentalHealthChatbot()

    print("\n🧠 CrewAI Mental Health Chatbot Ready!")
    print("Type 'exit' to quit, 'gradio' to launch web interface\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Take care! Remember, professional help is always available if you need it.")
            break
        elif user_input.lower() == 'gradio':
            print("Launching Gradio interface...")
            demo = create_gradio_interface()
            demo.launch()
            break

        print("🤖 Processing with multi-agent workflow...")
        response = chatbot.chat(user_input)
        print(f"Chatbot: {response}\n")

if __name__ == "__main__":
    # Set environment variable for Groq API key
    os.environ["GROQ_API_KEY"] = "YOUR_GROK_API_KEY"

    # Install required packages
    print("Installing required packages...")
    os.system("pip install crewai spacy textblob gradio langchain-groq chromadb sentence-transformers")
    os.system("python -m spacy download en_core_web_sm")

    main()


**TELEGRAM BOT**

In [ ]:
!pip install nest-asyncio python-telegram-bot
import nest_asyncio
nest_asyncio.apply()

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters



async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hello! I'm your mental health assistant. How can I help you today?")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text
    response = chatbot.chat(user_message)
    await update.message.reply_text(response)

def main():
    app = ApplicationBuilder().token("YOUR_TELEGRAM_TOKEN").build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    app.run_polling()

if __name__ == "__main__":
    main()